<img src="https://media.giphy.com/media/l41Y9SKrl3kLVamfC/giphy.gif">

#### Creo una API con personajes y frases de Stranger Things para analizar los sentimientos de los personajes y hacerles recomendaciones

In [28]:
import requests

### Insertar personajes en la base de datos

Se pueden insertar users en la db con una lista de diccionarios, o con un diccionario. 

In [5]:
personajes = [{"name":"Mike Wheeler"},
              {"name":"Once"},
              {"name":"Billy Hargrove"},
              {"name":"Dustin Henderson"},
              {"name":"Steve Harrington"},
              {"name":"Will"},
              {"name":"Suzie"},
              {"name":"Jonathan"},
              {"name":"Joyce Byers"},
              {"name":"Erica Sinclair"},
              {"name":"Karen Wheeler"},
              {"name":"Max"},
              {"name":"Bob Newby"},
              {"name":"Nancy Wheeler"},
              {"name":"Jim Hopper"},
              {"name":"Dr. Owens"},
              {"name":"Kali"},
              {"name":"Murray Bauman"},
              {"name":"Lucas"}
]

Para insertar usuarios en la base de datos hay que hacer una request de la siguiente manera. El sistema checkea si existen ya y les asigna un user_id correlativo. 

In [6]:
url_user = "http://localhost:5000/new/user"
for x in personajes:
    requests.post(url_user, data=x)

### Insertar chats/grupos en la base de datos

Para insertar chats en la base de datos, hay que hacerlo en forma de diccionario. Tiene que tener un nombre de diccionario y los participantes que están dentro. El sistema controlará que los participantes existan y que el nombre del chat no exista anteriormente.

In [19]:
friends = { "chat_name": "friends",
           "participants": ["Mike Wheeler","Dustin Henderson","Will","Lucas"]
}

hawkings = {"chat_name":"hawkins",
            "participants":['Mike Wheeler','Once','Billy Hargrove','Dustin Henderson','Steve Harrington','Will','Suzie','Jonathan','Joyce Byers','Erica Sinclair','Karen Wheeler','Max','Bob Newby','Nancy Wheeler','Jim Hopper','Dr. Owens','Kali','Murray Bauman','Lucas']   
}

friendsnew = {"chat_name" : "friends_new",
             "participants": ["Mike Wheeler","Once","Lucas","Dustin Henderson","Steve Harrington","Nancy Wheeler","Will","Suzie","Max"]
}
hopper_once = {"chat_name":"hopper_once", 
               "participants":["Once","Jim Hopper"]}
family_byers = {"chat_name": "family_byers",
               "participants": ["Will","Joyce Byers","Jonathan","Bob Newby"]}

In [26]:
url_chat = "http://localhost:5000/new/friendsnew"

In [27]:
requests.post(url_chat, data=friends)

<Response [404]>

### Insertar mensajes en la base de datos

Para insertar mensajes en la base de datos, el formato tiene que ser el siguiente. El sitema controlará que tanto el usuario como el chat existan. Además controlará que ese user esté dentro de ese chat y que ese mensaje no se haya dicho antes.

In [12]:
mensaje = {'name': 'Once', 'chat_name': 'friends_new', 'message': 'Los amigos no mienten'}

In [13]:
url_mensaje = "http://localhost:5000/new/message"
requests.post(url_mensaje, data=mensaje)

<Response [500]>

In [14]:
mensaje2 = {'name': 'Murray Bauman', 'chat_name': 'hawkins', 'message': 'Bueno, cuando acabéis de resolver vuestros problemas me avisáis para trabajar'}

In [15]:
url_mensaje = "http://localhost:5000/new/message"
requests.post(url_mensaje, data=mensaje2)

<Response [500]>

### Obtener info a través de la API
### Usuarios
¿Qué usuarios hay registrados? - Con esta requet vemos todos los users de la base de datos

In [16]:
url_users = "http://localhost:5000/users"

In [17]:
users = requests.get(url_users).json()
print(users[0:3])
print(len(users))

[{'name': ['Mike Wheeler']}, {'name': ['Once']}, {'name': ['Billy Hargrove']}]
19


¿En qué grupos está un user? - Con esta request vemos los grupos en los que está el user que queramos.

In [18]:
url_chats = "http://localhost:5000/user/chat/"
name = "Mike Wheeler"

In [5]:
mike = requests.get(url_chats + name).json()
print(mike)

[{'chat_name': 'friends'}, {'chat_name': 'hawkins'}, {'chat_name': 'friends_new'}]


### Grupos
¿Qué grupos de usuarios hay creados?

In [6]:
url_todos_chats = "http://localhost:5000/chats"

In [7]:
chats = requests.get(url_todos_chats).json()
print(chats)

[{'chat_name': 'friends'}, {'chat_name': 'hawkins'}, {'chat_name': 'friends_new'}, {'chat_name': 'hopper_once'}, {'chat_name': 'family_byers'}]


¿Qué usuarios hay en un grupo?

In [29]:
users_chat = "http://localhost:5000/chat/"
chat = "friends"

In [30]:
chat = requests.get(users_chat + chat).json()
chat

[{'chat_name': 'friends',
  'participants': ['Mike Wheeler', 'Dustin Henderson', 'Will', 'Lucas']}]

### Mensajes
¿Qué mensajes ha escrito un user? - Con esta request vemos los mensajes que ha escrito el user que solicitemos y en qué grupo los ha escrito. 

In [10]:
url_mensajes = "http://localhost:5000/user/message/"
name2 = "Once"

In [11]:
mensajes_once = requests.get(url_mensajes + name2).json()
mensajes_once[0]

{'name': 'Once',
 'chat_name': 'friends_new',
 'message': 'Los amigos no mienten'}

¿Qué mensajes se han escrito en un chat?

In [12]:
url_mensajes_chat = "http://localhost:5000/chat/message/"
grupo = "hawkins"

In [13]:
mensajes_hawkins= requests.get(url_mensajes_chat + grupo).json()
mensajes_hawkins[0:2]

[{'name': 'Bob Newby',
  'chat_name': 'hawkins',
  'message': 'Siempre pensé que este tipo de cosas ocurrían en las películas y las historietas.'},
 {'name': 'Joyce Byers',
  'chat_name': 'hawkins',
  'message': 'No me interesa si alguien me cree'}]

### Analizando los sentimientos de diferentes mensajes

##### Chat
Podemos analizar los sentimientos de un chat utilizando la siguiente ruta.

In [14]:
url_sentchat = "http://localhost:5000/chat/sentiments/"
grupo2 =  "hawkins"

In [15]:
sentimientos_hawkins= requests.get(url_sentchat + grupo2).json()
sentimientos_hawkins

{'polaridad': "0.018847815157655583['👍']",
 'subjetividad': '0.3183070254213871'}

##### Analizando Chat por Personajes
Puedes analizar la polaridad general del chat y ver cómo afecta la polaridad de cada personaje en la polaridad general del chat.

In [8]:
url_sent_chat_pers = "http://localhost:5000/chat/sentiments/users/"
gruporuser = "friends"

In [12]:
chatporuser = requests.get(url_sent_chat_pers + gruporuser).json()
chatporuser

[{'polaridad': "-0.013526272577996708['👎']",
  'subjetividad': '0.3044950738916256'},
 [{'Mike Wheeler': {'polaridad': "0.05['👍']", 'subjetividad': '0.2125'}},
  {'Dustin Henderson': {'polaridad': "0.011813186813186828['👍']",
    'subjetividad': '0.36227106227106226'}},
  {'Will': {'polaridad': "-0.23645833333333333['👎']",
    'subjetividad': '0.48020833333333335'}},
  {'Lucas': {'polaridad': "0.0['👍']", 'subjetividad': '0.125'}}]]

##### User - Un mensaje Random
Podemos analizar los sentimientos de un mensaje random de cualquier usuario

In [30]:
url_sentmensaje = "http://localhost:5000/user/random/sentiments/"
lucas =  "Lucas"

In [31]:
sentimientos_mensaje= requests.get(url_sentmensaje + lucas).json()
sentimientos_mensaje

{'Frase': 'No eres lo suficientemente ágil, pero un día lo serás. Pero hasta entonces… la Princesa Daphne aún es mía!',
 'polaridad': "-0.125['👎']",
 'subjetividad': '0.625'}

##### User
También podemos analizar los sentimientos de un user por los mensajes que ha enviado.

In [32]:
url_sentuser = "http://localhost:5000/user/sentiments/"
dustin ="Dustin Henderson"

In [33]:
sentimientos_dustin= requests.get(url_sentuser + dustin).json()
sentimientos_dustin

{'polaridad': "-0.0274035973084886['👎']",
 'subjetividad': '0.43438146997929605'}

Vemos que Dustin no es una persona tan positiva como parece, por las frases que dice....¿qué le pasará?

<img src="https://media.giphy.com/media/l3JDIP5O3Nz08HvJS/giphy.gif">

### NLP Sistema de recomendaciones
Vamos a recomendar usuarios por lo parecido de sus conversaciones en los chats.

In [2]:
url_recomendacion = "http://localhost:5000/recommend/"
name = "Jonathan"

In [3]:
recomendamos = requests.get(url_recomendacion + name).json()
recomendamos

{'Jonathan': 'Creo que conectas bastante bien con  Mike Wheeler'}